In [49]:
!pip install linearmodels
!pip install stargazer

import pandas as pd
import statsmodels.api as sm
import numpy as np 
from scipy import stats
from matplotlib import style
from matplotlib import pyplot as plt
from linearmodels.iv import IV2SLS
from linearmodels.iv import compare
import statsmodels.formula.api as smf
from stargazer.stargazer import Stargazer
from IPython.core.display import HTML
from scipy.stats import chi2
from tabulate import tabulate


In [50]:
#Table 1
data = pd.read_stata("C:/Users/sumit/Downloads/maketable1.dta")
data.dropna(inplace=True)
data.head()
data['mort_quartile'] = pd.qcut(data['extmort4'], q=4, labels=False)
data.head()

,shortnam,euro1900,avexpr,logpgp95,cons1,democ00a,cons00a,extmort4,logem4,loghjypl,baseco,mort_quartile
124,AGO,8.000000,5.363636,7.770645,3.0,0.0,1.0,280.000000,5.634789,-3.411248,1.0,3
130,ARG,60.000004,6.386364,9.133459,1.0,3.0,3.0,68.900002,4.232656,-0.872274,1.0,0
134,AUS,98.000000,9.318182,9.897972,7.0,10.0,7.0,8.550000,2.145931,-0.170788,1.0,0
140,BFA,0.000000,4.454545,6.845880,3.0,0.0,1.0,280.000000,5.634789,-3.540459,1.0,3
141,BGD,0.000000,5.136364,6.877296,7.0,0.0,1.0,71.410004,4.268438,-2.063568,1.0,1


In [51]:
world_mean = data[['logpgp95','loghjypl','avexpr','cons00a','cons1','democ00a','euro1900','logem4']].mean()
world_std = data[['logpgp95','loghjypl','avexpr','cons00a','cons1','democ00a','euro1900','logem4']].std()
base_sample = data[data['baseco'] == 1]
base_sample_mean = base_sample[['logpgp95','loghjypl','avexpr','cons00a','cons1','democ00a','euro1900','logem4']].mean()
base_sample_std = base_sample[['logpgp95','loghjypl','avexpr','cons00a','cons1','democ00a','euro1900','logem4']].std()
quartile_means = data.groupby('mort_quartile')[['logpgp95', 'loghjypl', 'avexpr', 'cons00a', 'cons1', 'democ00a', 'euro1900', 'logem4']].mean()

In [52]:
data = {
    'world_mean': world_mean,
    'world_std': world_std,
    'base_sample_mean': base_sample_mean,
    'base_sample_std': base_sample_std}
df = pd.DataFrame(data)
df.T


,logpgp95,loghjypl,avexpr,cons00a,cons1,democ00a,euro1900,logem4
world_mean,8.063693,-1.966828,6.511962,2.175439,3.491228,1.614035,17.287720,4.725914
world_std,0.977989,0.977983,1.514023,2.062616,2.413672,3.016350,26.512985,1.250925
base_sample_mean,8.063693,-1.966828,6.511962,2.175439,3.491228,1.614035,17.287720,4.725914
base_sample_std,0.977989,0.977983,1.514023,2.062616,2.413672,3.016350,26.512985,1.250925


In [53]:
quartile_means

,logpgp95,loghjypl,avexpr,cons00a,cons1,democ00a,euro1900,logem4
mort_quartile,,,,,,,,
0,8.866786,-1.187440,7.772727,3.666667,4.733333,3.800000,34.566666,3.353572
1,8.377945,-1.435024,6.533333,2.666667,2.266667,2.200000,22.933332,4.301359
2,7.632796,-2.318065,5.695804,1.153846,3.538461,0.153846,8.838462,5.040656
3,7.266655,-3.045529,5.896104,1.000000,3.428571,0.000000,0.571429,6.358899


In [54]:
# Table 2
table2_data = pd.read_stata("C:/Users/sumit/Downloads/maketable2.dta")
table2_data.dropna(inplace=True)
# Regression Analysis
# Column 1
model1 =sm.OLS.from_formula('logpgp95~avexpr',data= table2_data).fit()
# Column 2
model2 = sm.OLS.from_formula('logpgp95~avexpr',data= table2_data[table2_data['baseco'] == 1]).fit()

# Column 3
model3 = sm.OLS.from_formula('logpgp95~avexpr+lat_abst',data=table2_data).fit()

# Column 4
model4 = sm.OLS.from_formula('logpgp95~avexpr+lat_abst+africa+asia+other',data=table2_data).fit()

# Column 5
model5 = sm.OLS.from_formula('logpgp95~avexpr+lat_abst',data=table2_data[table2_data['baseco'] == 1]).fit()

# Column 6
model6 = sm.OLS.from_formula('logpgp95~avexpr+lat_abst+africa+asia+other',data=table2_data[table2_data['baseco'] == 1]).fit()

# Column 7
model7 = sm.OLS.from_formula('loghjypl~avexpr',data=table2_data).fit()

# Column 8
model8 = sm.OLS.from_formula('loghjypl~avexpr',data=table2_data[table2_data['baseco'] == 1]).fit()

print(model1.summary())
print(model2.summary())
print(model3.summary())
print(model4.summary())
print(model5.summary())
print(model6.summary())
print(model7.summary())
print(model8.summary())

                            OLS Regression Results                            
Dep. Variable:               logpgp95   R-squared:                       0.552
Model:                            OLS   Adj. R-squared:                  0.544
Method:                 Least Squares   F-statistic:                     72.66
Date:                Thu, 13 Mar 2025   Prob (F-statistic):           7.16e-12
Time:                        13:33:17   Log-Likelihood:                -62.874
No. Observations:                  61   AIC:                             129.7
Df Residuals:                      59   BIC:                             134.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      4.7824      0.398     12.028      0.0

In [55]:
# Render HTML
stargazer = Stargazer([model1, model2, model3, model4, model5, model6, model7, model8])
HTML(stargazer.render_html())

In [56]:
# Table 3
table3_data = pd.read_stata("C:/Users/sumit/Downloads/maketable3.dta")
table3_data.dropna(inplace=True)
table3_data = table3_data[table3_data['excolony']==1]
table3_data['euro1900']= table3_data['euro1900']/100
table3_data.columns
model19 = sm.OLS.from_formula('avexpr~cons00a',data=table3_data).fit()
model20 = sm.OLS.from_formula('avexpr~lat_abst+cons00a',data=table3_data).fit()
model21 = sm.OLS.from_formula('avexpr~democ00a',data=table3_data).fit()
model22 = sm.OLS.from_formula('avexpr~democ00a+lat_abst',data=table3_data).fit()
model23 = sm.OLS.from_formula('avexpr~indtime+cons1',data=table3_data).fit()
model24 = sm.OLS.from_formula('avexpr~indtime+cons1+lat_abst',data=table3_data).fit()
model25 = sm.OLS.from_formula('avexpr~euro1900',data=table3_data).fit()
model26 = sm.OLS.from_formula('avexpr~euro1900+lat_abst',data=table3_data).fit()
model27 = sm.OLS.from_formula('avexpr~logem4',data=table3_data.dropna(subset='logpgp95')).fit()
model27 = sm.OLS.from_formula('avexpr~logem4+lat_abst',data=table3_data.dropna(subset='logpgp95')).fit()

In [57]:
stargazer = Stargazer([model19, model20, model21, model22, model23, model24, model25, model26, model27])
HTML(stargazer.render_html())

In [58]:
# Table 4
table4_data =pd.read_stata("C:/Users/sumit/Downloads/maketable4.dta")
table4_data.dropna(inplace=True)
table4_data = table4_data[table4_data['baseco']==1]
table4_data.columns
c4_1 = 'logpgp95 ~ 1 + [avexpr~logem4]'
IV4_1 = IV2SLS.from_formula(c4_1, table4_data).fit()

c4_2 = 'logpgp95 ~ 1 + lat_abst + [avexpr~logem4]'
IV4_2 = IV2SLS.from_formula(c4_2, table4_data).fit()

table4_data_non_neo = table4_data[table4_data['rich4'] != 1]
c4_3 = 'logpgp95 ~ 1 + [avexpr~logem4]'
IV4_3 = IV2SLS.from_formula(c4_3, table4_data_non_neo).fit()

c4_4 = 'logpgp95 ~ 1 + lat_abst + [avexpr~logem4]'
IV4_4 = IV2SLS.from_formula(c4_4, table4_data_non_neo).fit()

table4_data_non_africa = table4_data[table4_data['africa'] != 1]
c4_5 = 'logpgp95 ~ 1 + [avexpr~logem4]'
IV4_5 = IV2SLS.from_formula(c4_5, table4_data_non_africa).fit()

c4_6 = 'logpgp95 ~ 1 + lat_abst + [avexpr~logem4]'
IV4_6 = IV2SLS.from_formula(c4_6, table4_data_non_africa).fit()

table4_data['other_cont'] = 0
table4_data.loc[table4_data['shortnam'].isin(["AUS", "MLT", "NZL"]), 'other_cont'] = 1

c4_7 = 'logpgp95 ~ 1 + [avexpr ~ logem4] + africa + asia + other_cont'
IV4_7 = IV2SLS.from_formula(c4_7, table4_data).fit()

c4_8 = 'logpgp95 ~ 1 + [avexpr ~ logem4] + africa + asia + other_cont + lat_abst'
IV4_8 = IV2SLS.from_formula(c4_8, table4_data).fit()

table4_data_with_loghjypl = table4_data[table4_data['loghjypl'].notnull()]
IV4_9 = IV2SLS.from_formula('loghjypl ~ 1 + [avexpr~logem4]', table4_data_with_loghjypl).fit()

Panel4A = [IV4_1, IV4_2, IV4_3, IV4_4, IV4_5, IV4_6, IV4_7, IV4_8, IV4_9]
COMPARE4A = compare({"1": IV4_1, "2": IV4_2, "3": IV4_3, "4": IV4_4, "5": IV4_5, "6": IV4_6, "7": IV4_7, "8": IV4_8, "9": IV4_9})
print('TABLE 4 PANEL A:', COMPARE4A)

TABLE 4 PANEL A:                                                                  Model Comparison                                                                
                                 1             2             3             4            5             6             7             8             9
-------------------------------------------------------------------------------------------------------------------------------------------------
Dep. Variable             logpgp95      logpgp95      logpgp95      logpgp95     logpgp95      logpgp95      logpgp95      logpgp95      loghjypl
Estimator                  IV-2SLS       IV-2SLS       IV-2SLS       IV-2SLS      IV-2SLS       IV-2SLS       IV-2SLS       IV-2SLS       IV-2SLS
No. Observations                61            61            57            57           35            35            61            61            61
Cov. Est.                   robust        robust        robust        robust       robust        robust    

In [59]:
# Table 4B
FS4_1b = IV2SLS.from_formula('avexpr ~ logem4', table4_data).fit()
# print(FS4_1b.summary())

FS4_2b = IV2SLS.from_formula('avexpr ~ logem4 + lat_abst', table4_data).fit()
# print(FS4_2b.summary())

FS4_3b = IV2SLS.from_formula('avexpr ~ 1 + logem4', table4_data[table4_data['rich4'] != 1]).fit()
# print(FS4_3b.summary())

FS4_4b = IV2SLS.from_formula('avexpr ~ 1 + logem4 + lat_abst', table4_data[table4_data['rich4'] != 1]).fit()
# print(FS4_4b.summary())

FS4_5b = IV2SLS.from_formula('avexpr ~ 1 + logem4', table4_data[table4_data['africa'] != 1]).fit()
# print(FS4_5b.summary())

FS4_6b = IV2SLS.from_formula('avexpr ~ logem4 + lat_abst', table4_data[table4_data['africa'] != 1]).fit()
# print(FS4_6b.summary())

FS4_7b = IV2SLS.from_formula('avexpr ~ logem4 + asia + africa + other_cont', table4_data).fit()
# print(FS4_7b.summary())

FS4_8b = IV2SLS.from_formula('avexpr ~ logem4 + lat_abst + asia + africa + other_cont', table4_data).fit()
# print(FS4_8b.summary())

FS4_9b = IV2SLS.from_formula('loghjypl ~ logem4', table4_data[table4_data['loghjypl'].notnull()]).fit()
# print(FS4_9b.summary())

# Place the models in a list
Panel4B = [FS4_1b, FS4_2b, FS4_3b, FS4_4b, FS4_5b, FS4_6b, FS4_7b, FS4_8b, FS4_9b]

# Assuming 'compare' is a function that compares models
COMPARE4B = compare({"1": FS4_1b, "2": FS4_2b, "3": FS4_3b, "4": FS4_4b, "5": FS4_5b, "6": FS4_6b, "7": FS4_7b, "8": FS4_8b, "9": FS4_9b})

# Print the comparison results
print('TABLE 4 PANEL B:', COMPARE4B)

TABLE 4 PANEL B:                                                        Model Comparison                                                       
                             1           2           3           4           5           6           7           8           9
------------------------------------------------------------------------------------------------------------------------------
Dep. Variable           avexpr      avexpr      avexpr      avexpr      avexpr      avexpr      avexpr      avexpr    loghjypl
Estimator                  OLS         OLS         OLS         OLS         OLS         OLS         OLS         OLS         OLS
No. Observations            61          61          57          57          35          35          61          61          61
Cov. Est.               robust      robust      robust      robust      robust      robust      robust      robust      robust
R-squared               0.2873      0.3051      0.1409      0.1426      0.4810      0.4857    

In [60]:
#Panel C, OLS Regressions
#Columns 1 - 2 (Base Sample)
model50 = sm.OLS.from_formula('logpgp95~avexpr',data=table4_data).fit()
model51 = sm.OLS.from_formula('logpgp95~lat_abst+avexpr',data=table4_data).fit()
#Columns 3 - 4 (Base Sample w/o Neo-Europes)
model53 = sm.OLS.from_formula('logpgp95~avexpr',data=table4_data[table4_data['rich4']!=1]).fit()
model54 = sm.OLS.from_formula('logpgp95~lat_abst+avexpr',data=table4_data[table4_data['rich4']!=1]).fit()
#Columns 5 - 6 (Base Sample w/o Africa)
model55 = sm.OLS.from_formula('logpgp95~avexpr',data=table4_data[table4_data['africa']!=1]).fit()
model56 = sm.OLS.from_formula('logpgp95~lat_abst+avexpr',data=table4_data[table4_data['africa']!=1]).fit()
#Columns 7 - 8 (Base Sample with continent dummies)
model57 = sm.OLS.from_formula('logpgp95~avexpr+africa+asia+other_cont',data=table4_data).fit()
model58 = sm.OLS.from_formula('logpgp95~lat_abst+avexpr+africa+asia +other_cont',data= table4_data).fit()
#Column 9 (Base Sample, log GDP per worker)
model59 = sm.OLS.from_formula('loghjypl~avexpr',data= table4_data).fit()
Panel4A = [model50, model51, model53, model54, model55, model56, model57, model58, model59]

stargazer = Stargazer(Panel4A)  # List of models
HTML(stargazer.render_html())

In [63]:
# Table 5
table5_data = pd.read_stata("C:/Users/sumit/Downloads/maketable5.dta")
table5_data.dropna(inplace=True)
table5_data.columns
table5_data = table5_data[table5_data['baseco']==1]
#Panel A 

model61 = IV2SLS.from_formula('logpgp95 ~ 1 + [avexpr ~ logem4] + f_brit+ f_french', data=
                              pd.concat([table5_data, pd.get_dummies(table5_data[['f_brit', 'f_french']], drop_first=True)], axis=1)).fit()
model62 = IV2SLS.from_formula('logpgp95 ~ 1 + lat_abst+[avexpr ~ logem4] + f_brit+ f_french', data=pd.concat([table5_data, pd.get_dummies(table5_data[['f_brit', 'f_french']], drop_first=True)], axis=1)).fit()
#Columns 3 and 4 (British colonies only)
model63 = IV2SLS.from_formula('logpgp95 ~ 1 + [avexpr ~ logem4] + sjlofr', data=pd.concat([table5_data, 
                         pd.get_dummies(table5_data['sjlofr'], drop_first=True)], axis=1)).fit()
model64 = IV2SLS.from_formula('logpgp95 ~ 1 + lat_abst+[avexpr ~ logem4] + sjlofr', data=pd.concat([table5_data, 
                         pd.get_dummies(table5_data['sjlofr'], drop_first=True)], axis=1)).fit()
#Columns 7 and 8 (Religion dummies)
model65 = IV2SLS.from_formula('logpgp95 ~ 1 + [avexpr ~ logem4] + catho80 + muslim80 + no_cpm80',
                               data=pd.concat([table5_data, 
                                               pd.get_dummies(table5_data[['catho80', 'muslim80', 'no_cpm80']], drop_first=True)], axis=1)).fit()

# IV regression 2
model66 = IV2SLS.from_formula('logpgp95 ~ 1 + lat_abst + [avexpr ~ logem4] + catho80 + muslim80 + no_cpm80',
                               data=pd.concat([table5_data, 
                                               pd.get_dummies(table5_data[['catho80', 'muslim80', 'no_cpm80']], drop_first=True)], axis=1)).fit()
#Columns 9 (Multiple controls)
# IV regression 1
model67 = IV2SLS.from_formula('logpgp95 ~ 1 + [avexpr ~ logem4] + f_french + sjlofr + catho80 + muslim80 + no_cpm80',
                               data=pd.concat([table5_data,
                                               pd.get_dummies(table5_data[['f_french', 'sjlofr', 'catho80', 'muslim80', 'no_cpm80']], drop_first=True)], axis=1)).fit()

# IV regression 2
model68 = IV2SLS.from_formula('logpgp95 ~ 1 + lat_abst + [avexpr ~ logem4] + f_french + sjlofr + catho80 + muslim80 + no_cpm80',
                               data=pd.concat([table5_data, 
                                               pd.get_dummies(table5_data[['f_french', 'sjlofr', 'catho80', 'muslim80', 'no_cpm80']], drop_first=True)], axis=1)).fit()
# Apply compare function to IV regressions in Panel B
COMPARE5A = compare({"1": model61,"2": model62,"3": model63,"4": model64,"5": model65,"6": model66,"7": model67,"8": model68})

# Print the comparison results
print('TABLE 5 PANEL A:', COMPARE5A)

TABLE 5 PANEL A:                                                            Model Comparison                                                           
                                   1             2             3             4             5             6             7             8
--------------------------------------------------------------------------------------------------------------------------------------
Dep. Variable               logpgp95      logpgp95      logpgp95      logpgp95      logpgp95      logpgp95      logpgp95      logpgp95
Estimator                    IV-2SLS       IV-2SLS       IV-2SLS       IV-2SLS       IV-2SLS       IV-2SLS       IV-2SLS       IV-2SLS
No. Observations                  64            64            64            64            64            64            64            64
Cov. Est.                     robust        robust        robust        robust        robust        robust        robust        robust
R-squared                     0.0483  

In [ ]:
#Panel C
model69_iv = IV2SLS.from_formula('logpgp95 ~ 1 + [avexpr ~ logem4] + f_brit + f_french', data=table5_data).fit()
model70_iv = IV2SLS.from_formula('logpgp95 ~ 1 + lat_abst + [avexpr ~ logem4] + f_brit + f_french', data=table5_data).fit()
model71_iv = IV2SLS.from_formula('logpgp95 ~ 1 + [avexpr ~ logem4]', data=table5_data[table5_data['f_brit'] == 1]).fit()
model72_iv = IV2SLS.from_formula('logpgp95 ~ 1 + lat_abst + [avexpr ~ logem4]', data=table5_data[table5_data['f_brit'] == 1]).fit()
model73_iv = IV2SLS.from_formula('logpgp95 ~ 1 + [avexpr ~ logem4] + sjlofr', data=table5_data).fit()
model74_iv = IV2SLS.from_formula('logpgp95 ~ 1 + lat_abst + [avexpr ~ logem4] + sjlofr', data=table5_data).fit()
model75_iv = IV2SLS.from_formula('logpgp95 ~ 1 + [avexpr ~ logem4] + catho80 + muslim80 + no_cpm80', data=table5_data).fit()
model76_iv = IV2SLS.from_formula('logpgp95 ~ 1 + lat_abst + [avexpr ~ logem4] + catho80 + muslim80 + no_cpm80', data=table5_data).fit()
model77_iv = IV2SLS.from_formula('logpgp95 ~ 1 + lat_abst + [avexpr ~ logem4] + f_french + sjlofr + catho80 + muslim80 + no_cpm80', data=table5_data).fit()

COMPARE5C_iv = compare({ "1": model69_iv, "2": model70_iv,"3": model71_iv,"4": model72_iv,"5": model73_iv,"6": model74_iv,"7": model75_iv,"8": model76_iv,"9": model77_iv})
print('TABLE 5 PANEL C (IV):', COMPARE5C_iv)


In [ ]:
# Table 6
table6_data = pd.read_stata("C:/Users/sumit/Downloads/maketable6.dta"")

table6_data = table6_data[table6_data['baseco']==1]
table6_data.columns
df6 = table6_data[table6_data['baseco']==1]
df6 = sm.add_constant(df6, has_constant="add")

# Define instrumental variable regressions

# Models for logpgp95 as the dependent variable
res_25 = IV2SLS(df6["logpgp95"], df6[['const','temp1', 'temp2', 'temp3', 'temp4', 'temp5', 'humid1', 'humid2', 'humid3', 'humid4']], df6["avexpr"], df6["logem4"]).fit(cov_type="unadjusted")
res_26 = IV2SLS(df6["logpgp95"], df6[['const','lat_abst','temp1', 'temp2', 'temp3', 'temp4', 'temp5', 'humid1', 'humid2', 'humid3', 'humid4']], df6["avexpr"], df6["logem4"]).fit(cov_type="unadjusted")
res_27 = IV2SLS(df6["logpgp95"], df6[["const","edes1975"]], df6["avexpr"], df6["logem4"]).fit(cov_type="unadjusted")
# Add more models as needed

# Models for avexpr as the dependent variable
res_34 = IV2SLS(df6["avexpr"], df6[['const','temp1', 'temp2', 'temp3', 'temp4', 'temp5', 'humid1', 'humid2', 'humid3', 'humid4','logem4']], None, None).fit(cov_type="unadjusted")
res_35 = IV2SLS(df6["avexpr"], df6[['const','lat_abst','temp1', 'temp2', 'temp3', 'temp4', 'temp5', 'humid1', 'humid2', 'humid3', 'humid4','logem4']], None, None).fit(cov_type="unadjusted")
res_36 = IV2SLS(df6["avexpr"], df6[["const","edes1975","logem4"]], None, None).fit(cov_type="unadjusted")
# Add more models as needed

# Models for logpgp95 with avexpr as an additional control
res_43 = IV2SLS(df6["logpgp95"], df6[['const','temp1', 'temp2', 'temp3', 'temp4', 'temp5', 'humid1', 'humid2', 'humid3', 'humid4','avexpr']], None, None).fit(cov_type="unadjusted")
res_44 = IV2SLS(df6["logpgp95"], df6[['const','lat_abst','temp1', 'temp2', 'temp3', 'temp4', 'temp5', 'humid1', 'humid2', 'humid3', 'humid4','avexpr']], None, None).fit(cov_type="unadjusted")
res_45 = IV2SLS(df6["logpgp95"], df6[["const","edes1975","avexpr"]], None, None).fit(cov_type="unadjusted")
# Add more models as needed

# Store the models in dictionaries
models_dict_logpgp95 = {
    "Model 1": res_25,
    "Model 2": res_26,
    "Model 3": res_27}

models_dict_avexpr = {
    "Model 1": res_34,
    "Model 2": res_35,
    "Model 3": res_36}

models_dict_logpgp95_with_avexpr = {
    "Model 1": res_43,
    "Model 2": res_44,
    "Model 3": res_45}

# Compare the models
comparison_results_logpgp95 = compare(models_dict_logpgp95)
comparison_results_avexpr = compare(models_dict_avexpr)
comparison_results_logpgp95_with_avexpr = compare(models_dict_logpgp95_with_avexpr)

# Print the comparison results
print("Comparison results for logpgp95:")
print(comparison_results_logpgp95)
print("\nComparison results for avexpr:")
print(comparison_results_avexpr)
print("\nComparison results for logpgp95 with avexpr:")
print(comparison_results_logpgp95_with_avexpr)

In [ ]:
# Table 7
table7_data_clean = pd.read_stata(r"C:\Users\shrey\Desktop\IIT DELHI\SEM 2\Econ lab\Assignment\maketable7.dta")
table7_data_clean.columns
table7_data_clean = table7_data_clean.dropna(subset=['malfal94', 'leb95', 'imr95', 'latabs', 'lt100km', 'meantemp'])
table7_data_clean = table7_data_clean[table7_data_clean['baseco']==1]
# Panels A and B, IV Regressions
model91 = IV2SLS.from_formula('logpgp95 ~ 1 + [avexpr ~ logem4] + malfal94', data=table7_data_clean).fit()
model92 = IV2SLS.from_formula('logpgp95 ~ 1 + lat_abst + [avexpr ~ logem4] + malfal94', data=table7_data_clean).fit()

# Columns 3 and 4: Instrumenting only for average protection against expropriation risk: life expectancy control
model93 = IV2SLS.from_formula('logpgp95 ~ 1 + [avexpr ~ logem4] + leb95', data=table7_data_clean).fit()
model94 = IV2SLS.from_formula('logpgp95 ~ 1 + lat_abst + [avexpr ~ logem4] + leb95', data=table7_data_clean).fit()

model95 = IV2SLS.from_formula('logpgp95 ~ 1 + [avexpr ~ logem4] + imr95', data=table7_data_clean).fit()
model96 = IV2SLS.from_formula('logpgp95 ~ 1 + lat_abst + [avexpr ~ logem4] + imr95', data=table7_data_clean).fit()

models_table7 = [model91, model92, model93, model94, model95, model96]

comparison = compare(models_table7)
print(comparison)

# Column 7: Instrumenting for all RHS variables: malaria control
model97 = IV2SLS.from_formula('logpgp95 ~ 1 + [avexpr+malfal94~logem4+latabs+lt100km+meantemp]', data=table7_data_clean).fit()

# Column 8: Instrumenting for all RHS variables: life expectancy control
model98 = IV2SLS.from_formula('logpgp95 ~ 1 + [avexpr+leb95~logem4+latabs+lt100km+meantemp]', data=table7_data_clean).fit()

# Column 9: Instrumenting for all RHS variables: infant mortality control
model99 = IV2SLS.from_formula('logpgp95 ~ 1 + [avexpr+imr95~logem4+latabs+lt100km+meantemp]', data=table7_data_clean).fit()

# Columns 10 and 11: Yellow fever instrument for average protection against expropriation risk
model200 = IV2SLS.from_formula('logpgp95 ~ 1 + [avexpr~yellow]', data=table7_data_clean).fit()
compare = compare({'Model 97': model97, 'Model 98': model98, 'Model 99': model99, 'Model 200': model200})
print(compare)
# Create other_cont variable
table7_data_clean['other_cont'] = 0
table7_data_clean.loc[table7_data_clean['shortnam'].isin(["AUS", "MLT", "NZL"]), 'other_cont'] = 1

# Perform instrumental variable regression
model201 = IV2SLS.from_formula('logpgp95 ~ 1 + [avexpr~yellow] + africa + asia + other_cont', data=table7_data_clean, weights=None).fit()
print(model201.summary)

In [ ]:
# Panel C, OLS Regressions
# Columns 1 and 2: Instrumenting only for average protection against expropriation risk: malaria control
model202 = smf.ols('logpgp95 ~ avexpr + malfal94', data=table7_data_clean).fit()
model203 = smf.ols('logpgp95 ~ lat_abst + avexpr + malfal94', data=table7_data_clean).fit()

# Columns 3 and 4: Instrumenting only for average protection against expropriation risk: life expectancy control
model204 = smf.ols('logpgp95 ~ avexpr + leb95', data=table7_data_clean).fit()
model205 = smf.ols('logpgp95 ~ lat_abst + avexpr + leb95', data=table7_data_clean).fit()

# Columns 5 and 6: Instrumenting only for average protection against expropriation risk: infant mortality control
model206 = smf.ols('logpgp95 ~ avexpr + imr95', data=table7_data_clean).fit()
model207 = smf.ols('logpgp95 ~ lat_abst + avexpr + imr95', data=table7_data_clean).fit()

# Column 7: Instrumenting for all RHS variables: malaria control
model208 = smf.ols('logpgp95 ~ avexpr + malfal94 + latabs + lt100km + meantemp', data=table7_data_clean).fit()

# Column 8: Instrumenting for all RHS variables: life expectancy control
model209 = smf.ols('logpgp95 ~ avexpr + leb95 + latabs + lt100km + meantemp', data=table7_data_clean).fit()

# Column 9: Instrumenting for all RHS variables: infant mortality control
model210 = smf.ols('logpgp95 ~ avexpr + imr95 + latabs + lt100km + meantemp', data=table7_data_clean).fit()
# List of models
models_table7 = [model91, model92, model93, model94, model95, model96, model97, model98, model99, model200, model201, model202, model203, model204, model205, model206, model207, model208, model209, model210]
stargazer = Stargazer(models_table7)

HTML(stargazer.render_html())


In [ ]:
# Table 8
table8_data_clean = pd.read_stata(r"C:\Users\shrey\Desktop\IIT DELHI\SEM 2\Econ lab\Assignment\maketable8.dta")
table8_data_clean.columns
# Drop null values for each variable
table8_data_clean.dropna(subset=['shortnam', 'lat_abst', 'euro1900', 'avexpr', 'logpgp95', 'democ1', 'cons1', 'indtime', 'democ00a', 'cons00a', 'logem4', 'baseco'], inplace=True)
table8_data_clean = table8_data_clean[table8_data_clean['baseco']==1]
#Panels A and B, IV Regressions
# Columns 1 - 2 (European settlements in 1900 as instrument)
model211 = IV2SLS.from_formula('logpgp95 ~ 1 + [avexpr~euro1900]', data=table8_data_clean).fit()
model212 = IV2SLS.from_formula('logpgp95 ~ lat_abst + [avexpr~euro1900]', data=table8_data_clean).fit()

# Columns 3 - 4 (Constraint on executive in 1900 as instrument)
model213 = IV2SLS.from_formula('logpgp95 ~ 1 + [avexpr~cons00a]', data=table8_data_clean).fit()
model214 = IV2SLS.from_formula('logpgp95 ~ lat_abst + [avexpr~cons00a]', data=table8_data_clean).fit()

# Columns 5 - 6 (Democracy in 1900)
model215 = IV2SLS.from_formula('logpgp95 ~ 1 + [avexpr~democ00a]', data=table8_data_clean).fit()
model216 = IV2SLS.from_formula('logpgp95 ~ lat_abst + [avexpr~democ00a]', data=table8_data_clean).fit()

# Columns 7 - 8 (Constraint on executive in first year of independence)
model217 = IV2SLS.from_formula('logpgp95 ~ 1 + [avexpr~cons1] + indtime', data=table8_data_clean).fit()
model218 = IV2SLS.from_formula('logpgp95 ~ lat_abst + [avexpr~cons1] + indtime', data=table8_data_clean).fit()

# Columns 9 - 10 (Democracy in first year of independence)
model219 = IV2SLS.from_formula('logpgp95 ~ 1 + [avexpr~democ1] + indtime', data=table8_data_clean).fit()
model220 = IV2SLS.from_formula('logpgp95 ~ lat_abst + [avexpr~democ1] + indtime', data=table8_data_clean).fit()
# List of models
models_table8 = [model211, model212, model213, model214, model215, model216, model217, model218, model219, model220]

from linearmodels.iv import compare  
comparison = compare(models_table8)


# Print the comparison results
print(comparison)


In [ ]:
# Panel C
# Columns 1 - 2 (European settlements in 1900 as instrument)
model221 = IV2SLS.from_formula('logpgp95 ~ 1 + [avexpr~euro1900] + logem4', data=table8_data_clean).fit()
model222 = IV2SLS.from_formula('logpgp95 ~ [avexpr~euro1900] + logem4', data=table8_data_clean).fit()

# Columns 3 - 4 (Constraint on executive in 1900 as instrument)
model223 = IV2SLS.from_formula('logpgp95 ~ 1 + [avexpr~cons00a] + logem4', data=table8_data_clean).fit()
model224 = IV2SLS.from_formula('logpgp95 ~ [avexpr~cons00a] + logem4', data=table8_data_clean).fit()

# Columns 5 - 6 (Democracy in 1900)
model225 = IV2SLS.from_formula('logpgp95 ~ 1 + [avexpr~democ00a] + logem4', data=table8_data_clean).fit()
model226 = IV2SLS.from_formula('logpgp95 ~ [avexpr~democ00a] + logem4', data=table8_data_clean).fit()

# Columns 7 - 8 (Constraint on executive in first year of independence)
model227 = IV2SLS.from_formula('logpgp95 ~ 1 + [avexpr~cons1] + indtime + logem4', data=table8_data_clean).fit()
model228 = IV2SLS.from_formula('logpgp95 ~ [avexpr~cons1] + indtime + logem4', data=table8_data_clean).fit()

# Columns 9 - 10 (Democracy in first year of independence)
model229 = IV2SLS.from_formula('logpgp95 ~ 1 + [avexpr~democ1] + indtime + logem4', data=table8_data_clean).fit()
model230 = IV2SLS.from_formula('logpgp95 ~ [avexpr~democ1] + indtime + logem4', data=table8_data_clean).fit()
# List of models
models_panelC = [model221, model222, model223, model224, model225, model226, model227, model228, model229, model230]

comparison1= compare(models_panelC)
print(comparison1)
